# Import Module

In [20]:
import numpy as np
import pandas as pd

# Load Dataset and Get Info

In [21]:
order_brush_order = pd.read_csv('order_brush_order.csv')

In [22]:
order_brush_order.head()

,orderid,shopid,userid,event_time
0,31076582227611,93950878,30530270,2019-12-27 00:23:03
1,31118059853484,156423439,46057927,2019-12-27 11:54:20
2,31123355095755,173699291,67341739,2019-12-27 13:22:35
3,31122059872723,63674025,149380322,2019-12-27 13:01:00
4,31117075665123,127249066,149493217,2019-12-27 11:37:55


In [23]:
order_brush_order.describe()

,orderid,shopid,userid
count,2.227500e+05,2.227500e+05,2.227500e+05
mean,3.130027e+13,9.433117e+07,9.802880e+07
std,1.222774e+11,5.695790e+07,6.839048e+07
min,3.107520e+13,1.000900e+04,1.000700e+04
25%,3.120360e+13,4.980267e+07,3.508127e+07
50%,3.130561e+13,9.033636e+07,9.309625e+07
75%,3.140604e+13,1.475053e+08,1.590612e+08
max,3.150720e+13,2.154352e+08,2.155262e+08


In [24]:
order_brush_order['shopid'].nunique()

18770

In [25]:
order_brush_order['userid'].nunique()

93883

In [26]:
order_brush_order.groupby('shopid')['orderid'].count().mean()

11.867341502397442

# Get Minimum Time

In [27]:
order_brush_order[order_brush_order['event_time'] == order_brush_order['event_time'].min()]

,orderid,shopid,userid,event_time
25985,31075200506752,104804492,97707522,2019-12-27 00:00:00
81020,31075200506753,8715449,97707522,2019-12-27 00:00:00
150060,31075200506751,6042309,97707522,2019-12-27 00:00:00


# Set event_time as index and Grouby

In [28]:
order_brush_order_time = order_brush_order.set_index(pd.DatetimeIndex(order_brush_order['event_time'])).drop('event_time', axis=1)
order_brush_order_time = order_brush_order_time.sort_index()

grouped_orders = order_brush_order_time.groupby(['shopid', 'userid', pd.Grouper(freq='60min', label='left', base=0)]).count()

# Find shopid that possibly do order brushing, given a threshold

In [29]:
possible_brush = grouped_orders[grouped_orders.orderid > 2]

userids = []
possible_brush.reset_index().groupby('shopid')['userid'].apply(lambda x: userids.append(x.values))
len(userids)

296

In [30]:
possible_brush.reset_index().shopid.nunique()

296

In [31]:
def concat_and(arr):
    res = '&'.join(str(x) for x in arr)
    return res

concat_userids = []
for element in userids:
    concat_userids.append(concat_and(element))

res_order_brush_order = pd.DataFrame({"shopid": possible_brush.reset_index()['shopid'].unique(), "userid": concat_userids})
res_order_brush_order.head()

,shopid,userid
0,10159,214988798
1,10287,198097381&214226569
2,10402,77819
3,10536,672345&672345
4,29583,214495220&215275495


In [32]:
default_order_brush_order = pd.DataFrame({'shopid': order_brush_order['shopid'].unique(), 'userid': 0})
default_order_brush_order.head()

,shopid,userid
0,93950878,0
1,156423439,0
2,173699291,0
3,63674025,0
4,127249066,0


In [33]:
res_order_brush_order.head()

,shopid,userid
0,10159,214988798
1,10287,198097381&214226569
2,10402,77819
3,10536,672345&672345
4,29583,214495220&215275495


In [34]:
res_order_brush_order.shape

(296, 2)

# Export Result

In [35]:
sol_order_brush_order = pd.concat([default_order_brush_order[~default_order_brush_order.shopid.isin(res_order_brush_order.shopid)], res_order_brush_order])
sol_order_brush_order.to_csv("solution.csv", index=False)

# Check result

In [36]:
pd.read_csv("solution.csv")

,shopid,userid
0,93950878,0
1,156423439,0
2,173699291,0
3,63674025,0
4,127249066,0
...,...,...
18765,203531250,114282846
18766,204225676,198662175
18767,208696908,214111334
18768,210197928,52867898
